In [10]:
! pip install requests
! pip install pandas
! pip install numpy
! pip install datetime

DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.12/site-packages/acnportal-0.3.3-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.12/site-packages/acnportal-0.3.3-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.12/site-packages/acnportal-0.3.3-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.12/site-packages/acnportal-0.3.3-py3.12.egg is deprecated. pip 24.3 will

In [12]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime

# =============================================================================
# 1. LOAD THE DATA
# =============================================================================
df_2018 = pd.read_csv('/Users/huibvanderveen/Desktop/Pycharm Projects/ML_New/YEAR/sessions_filtered_2018.csv')
df_2019 = pd.read_csv('/Users/huibvanderveen/Desktop/Pycharm Projects/ML_New/YEAR/sessions_filtered_2019.csv')
df_2020 = pd.read_csv('/Users/huibvanderveen/Desktop/Pycharm Projects/ML_New/YEAR/sessions_filtered_2020.csv')
df_2021 = pd.read_csv('/Users/huibvanderveen/Desktop/Pycharm Projects/ML_New/YEAR/sessions_filtered_2021.csv')


In [18]:
 import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime

# =============================================================================
# 1. LOAD THE DATA
# =============================================================================
# Replace these file paths with your actual CSV paths
df_2018 = pd.read_csv('/Users/huibvanderveen/Desktop/Pycharm Projects/ML_New/YEAR/sessions_filtered_2018.csv')
df_2019 = pd.read_csv('/Users/huibvanderveen/Desktop/Pycharm Projects/ML_New/YEAR/sessions_filtered_2019.csv')
df_2020 = pd.read_csv('/Users/huibvanderveen/Desktop/Pycharm Projects/ML_New/YEAR/sessions_filtered_2020.csv')
df_2021 = pd.read_csv('/Users/huibvanderveen/Desktop/Pycharm Projects/ML_New/YEAR/sessions_filtered_2021.csv')

# A helper function to clean and process a raw DataFrame
def process_sessions_df(df_raw, year_label):
    # Drop rows with missing relevant data
    df = df_raw.dropna(subset=['sessionID', 'connectionTime',
                               'disconnectTime', 'doneChargingTime', 'kWhDelivered']).copy()

    # Convert to datetime
    for col in ['connectionTime', 'disconnectTime', 'doneChargingTime']:
        df[col] = pd.to_datetime(df[col], errors='coerce')

    # Drop rows with failed datetime conversion
    df.dropna(subset=['connectionTime', 'disconnectTime'], inplace=True)

    # Calculate session duration in hours
    df['session_duration'] = (df['disconnectTime'] - df['connectionTime']).dt.total_seconds() / 3600.0

    # Filter out non-positive and extreme durations (e.g., < 12 hours)
    df = df[(df['session_duration'] > 0) & (df['session_duration'] < 12)]

    # Normalize hour: shift so that 8:00 becomes 0 (times in GMT, garage is in LA)
    df['hour'] = (df['connectionTime'].dt.hour - 8) % 24

    # Calculate charging duration (time the vehicle actually charged) in hours
    df['charging_duration'] = (df['doneChargingTime'] - df['connectionTime']).dt.total_seconds() / 3600.0
    df = df[df['charging_duration'] > 0]

    # Calculate average power output in kW
    df['power_output'] = df['kWhDelivered'] / df['charging_duration']

    # Tag this DataFrame with a year column so we can merge them easily
    df['year'] = year_label
    return df

# Clean/process each yearly DataFrame
df_2018_clean = process_sessions_df(df_2018, 2018)
df_2019_clean = process_sessions_df(df_2019, 2019)
df_2020_clean = process_sessions_df(df_2020, 2020)
df_2021_clean = process_sessions_df(df_2021, 2021)

# Combine all years into one DataFrame for “arrival by hour” and ratio plots
df_all = pd.concat([df_2018_clean, df_2019_clean, df_2020_clean, df_2021_clean], ignore_index=True)

# =============================================================================
# 2. SESSION ARRIVAL BY HOUR (ALL YEARS, SEPARATE LINES)
# =============================================================================
# Group by year and hour, count number of sessions
arrival_rate = df_all.groupby(['year', 'hour']).size().reset_index(name='arrival_rate')

fig_arrival = px.line(
    arrival_rate,
    x='hour',
    y='arrival_rate',
    color='year',
    markers=True,
    labels={'hour': 'Hour of Day', 'arrival_rate': 'Number of Sessions'},
    title='Session Arrival Rate by Hour (all years)'
)
fig_arrival.update_layout(xaxis=dict(tickmode='linear', tick0=0, dtick=1))
fig_arrival.show()

# =============================================================================
# 3. RATIO OF SESSION DURATIONS BY HOUR (ALL YEARS)
#    Instead of simple average, we compute the fraction of total session-hours
#    that occur in each hour for each year.
# =============================================================================
# For each year-hour, sum up the durations
sum_durations = df_all.groupby(['year', 'hour'])['session_duration'].sum().reset_index(name='sum_duration')

# For each year separately, we find total session-hours and then the ratio
def compute_ratio(group):
    total = group['sum_duration'].sum()
    group['duration_ratio'] = group['sum_duration'] / total
    return group

sum_durations = sum_durations.groupby('year').apply(compute_ratio)

# Plot ratio_of_duration by hour, color by year
fig_ratio = px.line(
    sum_durations,
    x='hour',
    y='duration_ratio',
    color='year',
    markers=True,
    labels={'hour': 'Hour of Day', 'duration_ratio': 'Fraction of Total Session-Hours'},
    title='Ratio of Session Durations by Hour (all years)'
)
fig_ratio.update_layout(xaxis=dict(tickmode='linear', tick0=0, dtick=1))
fig_ratio.show()

# =============================================================================
# 4. AVERAGE SYSTEM DEMAND OVER THE YEAR, PER YEAR
#    We do the “power events” approach for each year and then combine results
#    so we can plot separate lines for each year in one figure.
# =============================================================================

# Function that, given a clean yearly DataFrame, returns daily average power
def compute_daily_avg_power(df_clean, year_label):
    # Create +power_output event at connectionTime
    df_start = df_clean[['connectionTime','power_output']].copy()
    df_start.rename(columns={'connectionTime': 'timestamp'}, inplace=True)

    # Create -power_output event at doneChargingTime
    df_end = df_clean[['doneChargingTime','power_output']].copy()
    df_end.rename(columns={'doneChargingTime': 'timestamp'}, inplace=True)
    df_end['power_output'] = -df_end['power_output']

    # Combine and sort
    events_df = pd.concat([df_start, df_end]).sort_values('timestamp')

    # Cumulative sum of power outputs to get instantaneous load
    events_df['total_power'] = events_df['power_output'].cumsum()

    # Resample by day, fill empty days with zero
    events_df = events_df.set_index('timestamp')
    daily_avg = (
        events_df['total_power']
        .resample('D')
        .mean()
        .fillna(0)
        .reset_index()
    )
    daily_avg['year'] = year_label
    return daily_avg

daily_avg_2018 = compute_daily_avg_power(df_2018_clean, 2018)
daily_avg_2019 = compute_daily_avg_power(df_2019_clean, 2019)
daily_avg_2020 = compute_daily_avg_power(df_2020_clean, 2020)
daily_avg_2021 = compute_daily_avg_power(df_2021_clean, 2021)

# Combine for plotting
daily_all = pd.concat([
    daily_avg_2018,
    daily_avg_2019,
    daily_avg_2020,
    daily_avg_2021
], ignore_index=True)

fig_demand = px.line(
    daily_all,
    x='timestamp',
    y='total_power',
    color='year',
    labels={'timestamp': 'Date', 'total_power': 'Average System Demand (kW)'},
    title='Average kW Demand of the Garage Throughout Each Year'
)
fig_demand.show()

# Print some quick summaries
print("Arrival Rate by Hour (by year, first few rows):")
print(arrival_rate.head())

print("\nRatio of Total Session-Hours by Hour (by year, first few rows):")
print(sum_durations.head())

print("\nDaily Average System Demand (kW) for each year (first few rows):")
print(daily_all.head())


/var/folders/v0/btbypqg16m19ftg41s8lglrm0000gn/T/ipykernel_57678/2961120123.py:89: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



Arrival Rate by Hour (by year, first few rows):
   year  hour  arrival_rate
0  2018     0            58
1  2018     1            21
2  2018     2            23
3  2018     3           136
4  2018     4            73

Ratio of Total Session-Hours by Hour (by year, first few rows):
        year  hour  sum_duration  duration_ratio
year                                            
2018 0  2018     0    228.405278        0.003181
     1  2018     1     72.245000        0.001006
     2  2018     2     77.466389        0.001079
     3  2018     3    942.560000        0.013126
     4  2018     4    261.582222        0.003643

Daily Average System Demand (kW) for each year (first few rows):
   timestamp  total_power  year
0 2018-04-25    34.177346  2018
1 2018-04-26    30.149907  2018
2 2018-04-27    30.727468  2018
3 2018-04-28    18.024874  2018
4 2018-04-29    19.785868  2018


In [21]:

# --- Compute average session duration by hour (per year) ---
avg_duration = df_all.groupby(['year', 'hour'])['session_duration'].mean().reset_index(name='avg_duration')

# --- Plot using Plotly Express ---
fig_avg_duration = px.line(
    avg_duration,
    x='hour',
    y='avg_duration',
    color='year',
    markers=True,
    labels={
        'hour': 'Hour of Day',
        'avg_duration': 'Average Session Duration (hours)',
        'year': 'Year'
    },
    title='Average Session Duration by Hour (per Year)'
)

fig_avg_duration.update_layout(
    xaxis=dict(tickmode='linear', tick0=0, dtick=1),
)

fig_avg_duration.show()